<a href="https://colab.research.google.com/github/AirPro/HandsOnMod3Freidrd/blob/main/HandsOnMod3Freidrd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 3 Hands On Assignment By: Robert Freid
It is in the hands-on. You need to watch the section 5 in the LinkedIn course which link is provided in the hands-on section. So, section 5 has different subtopic video list, you need to repeat exactly what has been done in these videos but using Google Colab.
The title of these subsections:

* Using a pre-trained network for object recognition
* Extracting features with a pre-trained neural network
* Training a new neural network with extracted features
* Making predictions with transfer learning


In [1]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import vgg16
import tensorflow as tf
print(tf.__version__)


2.11.0


In [2]:
!python --version

Python 3.8.10


## Using A Pre-Trained Network for Object Recognition
Repreduce the exercise from the LinkedIn video.

In [3]:
# Load the Keras VGG16 - Model is pretrained against the ImageNet database
model = vgg16.VGG16()

553467096/553467096 [==============================] - 7s 0us/step


In [4]:
# Upload the file from local computer: bay.jpg
from google.colab import files
uploaded = files.upload()

Saving bay.jpg to bay.jpg


In [5]:
# Load the Image file "bay.jpg" and resize it to 224 x 224 pixels
img = tf.keras.utils.load_img("bay.jpg", target_size=(224, 224))
# Convert image to numpy array
x = tf.keras.utils.img_to_array(img)

In [6]:
# Add a fourth dimension due to Karas expecting a list of images for consistency
x = np.expand_dims(x, axis=0)

In [7]:
# Normalize imput image values to range used for training nthe neural network
x = vgg16.preprocess_input(x)

In [8]:
# Initaite the convolutional neural network to make a prediction
predictions = model.predict(x)

1/1 [==============================] - 1s 1s/step


In [9]:
# Check for hthe names of the predicted classes. Index 0 is the results for the first image.
predicted_classes = vgg16.decode_predictions(predictions)

35363/35363 [==============================] - 0s 0us/step


In [10]:
# Print out the predictions for our test image
print("Top predictions for this image: ")
for imagenet_id, name, likelihood in predicted_classes[0]:
  print("Prediction: {} - {:2f}".format(name, likelihood) )

Top predictions for this image: 
Prediction: seashore - 0.395213
Prediction: promontory - 0.326129
Prediction: lakeside - 0.119613
Prediction: breakwater - 0.062801
Prediction: sandbar - 0.045267


## Extracting features with a pre-trained neural network
From Part 2 of the LinkedIn Hands-On Assignment

In [11]:
# import necessary libraaries and modules
from pathlib import Path 
import joblib 
# all other libraries and modules have been impoted from previous application

### Dealilng with importing the data into Google colab

In [12]:
# Import the zipped folder with the training data in it.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# unzip folder for access from program application
!unzip -uq "/content/drive/MyDrive/Google Drive/training_data.zip" -d ""

In [14]:
import keras.utils as image
import glob

# Path to folders with training data
# Path to is dogs: /content/training_data/dogs
# Path /content/training_data/not_dogs

dog_path = Path("/content/training_data/dogs/") # / "dogs"

print(dog_path)

not_dog_path = Path("/content/training_data/not_dogs/") # / "not_dogs"

print(not_dog_path)

# Declare arrays
images = []
labels = []



/content/training_data/dogs
/content/training_data/not_dogs


### Load All No Dog Images

In [15]:
# load not-dog images

# print("Pre-Loop Execution")
for img_nd in not_dog_path.glob("*.png"):

  # remove ._ from file extension
  img_nd = str(img_nd)
  img_nd = img_nd.replace('._', "")

  # Load image from disk storage
  img_nd = tf.keras.utils.load_img(img_nd)

  # Convert the image to a numpy array
  image_array =  tf.keras.utils.img_to_array(img_nd)

  # add image to list of images in the images array
  images.append(image_array)

  # for each "not dog" image, the expected value is zero
  labels.append(0)

  # print(img_nd)

### Load all the dog images

In [16]:
# load dog images
for img_dog in dog_path.glob("*.png"): 

  # remove ._ from file extension
  img_dog = str(img_dog)
  img_dog = img_dog.replace('._', "")

  # Load image from disk storage
  img_dog = tf.keras.utils.load_img(img_dog)

  # Convert the image to a numpy array
  image_array =  tf.keras.utils.img_to_array(img_dog)

  # add image to list of images in the images array
  images.append(image_array)

  # for each "not dog" image, the expected value is zero
  labels.append(1)

# print(img_dog)

### Create a simple numpy array with all inages we loaded

In [17]:
array_count = np.count_nonzero(images)
print(array_count)
x_train = np.array(images)

1402814


### Also create a simple numpy array for the labels

In [18]:
y_train = np.array(labels)

### Normalize the image data to a 0 to 1 range

In [19]:
x_train = vgg16.preprocess_input(x_train)

### Load a pre-trained neural network to use as a feature extractor

In [20]:
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3)) 

58889256/58889256 [==============================] - 0s 0us/step


### Extract features for each image (All in one pass)

In [21]:
features_x = pretrained_nn.predict(x_train) 

4/4 [==============================] - 7s 2s/step


### Save the array of the extracted features to a file

In [22]:
joblib.dump(features_x, "x_train.dat")

['x_train.dat']

### Save the matching array of expected values to a file.

In [23]:
joblib.dump(y_train, "y_train.dat")

['y_train.dat']

# Training a new neural network with extracted features
Load relevant libraries and modules

In [24]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten 
from pathlib import Path 
import joblib 

### Load Data Set

In [29]:
x_train = joblib.load("/content/x_train.dat") 
y_train = joblib.load("/content/y_train.dat")

### Create a model and add layers

In [31]:
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:])) 
model.add(Dense(256, activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(1, activation='sigmoid'))

### Compile the model

In [32]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

### Train the model

In [33]:
model.fit(
    x_train,
    y_train,
    epochs=10,
    shuffle=True
)

Epoch 1/10
4/4 [==============================] - 1s 13ms/step - loss: 10.6194 - accuracy: 0.6897
Epoch 2/10
4/4 [==============================] - 0s 11ms/step - loss: 0.3887 - accuracy: 0.9741
Epoch 3/10
4/4 [==============================] - 0s 9ms/step - loss: 0.0649 - accuracy: 0.9741
Epoch 4/10
4/4 [==============================] - 0s 8ms/step - loss: 0.0557 - accuracy: 0.9828
Epoch 5/10
4/4 [==============================] - 0s 12ms/step - loss: 1.4143e-08 - accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 0s 9ms/step - loss: 3.5335e-14 - accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 0s 9ms/step - loss: 2.1934e-08 - accuracy: 1.0000
Epoch 8/10
4/4 [==============================] - 0s 9ms/step - loss: 2.0643e-14 - accuracy: 1.0000
Epoch 9/10
4/4 [==============================] - 0s 9ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 10/10
4/4 [==============================] - 0s 11ms/step - loss: 2.9675e-08 - accuracy: 1.0000


### Save the Neural Network structure

In [34]:
model_structure = model.to_json()
f = Path("model_structure.json") 
f.write_text(model_structure)

1452

### Save the Neural Network Trained Weights

In [35]:
model.save_weights("model_weights.h5")

# Making predictions with transfer learning
Load modules and libraries needed to make predictions

In [36]:
from keras.models import model_from_json 
from pathlib import Path
import numpy as np
from keras.applications import vgg16

### Import the json file that contains the model structure

In [37]:
f = Path("model_structure.json") 
model_structure = f.read_text()

### Reload the models trained weights

In [38]:
model.load_weights("model_weights.h5")

### Load the image file to test, resizing it to 64 x 64 pixels
As required by this model

In [39]:
# Upload the file from local computer: not_dog.png
from google.colab import files
uploaded = files.upload()

Saving not_dog.png to not_dog.png


In [40]:
# Load the Image file "not_dog.png" and resize it to 64 x 64 pixels
img = tf.keras.utils.load_img("not_dog.png", target_size=(64, 64))
# Convert image to numpy array
image_array = tf.keras.utils.img_to_array(img)

### Add a fourth dimension to image 
Keras expects a bunch of images, not a sinlge layer 

In [41]:
images = np.expand_dims(image_array, axis=0) 

### Normalize the data

In [42]:
images = vgg16.preprocess_input(images)

### Use the pre-trained neural network to extract features from our test image.
The same way we did to train the model

In [43]:
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3)) 
features = feature_extraction_model.predict(images)

1/1 [==============================] - 0s 223ms/step


### Given the extracted features, make a final prediction using our own model.

In [44]:
results = model.predict(features) 

1/1 [==============================] - 0s 65ms/step


### Since we are only testing one image with possible class:
We only need to check the first result's first element

In [45]:
single_result = results[0] [0] 

### Print the result!

In [46]:
print("Likelihood that this image contains a dog: {}%".format(int(single_result * 100)))

Likelihood that this image contains a dog: 0%
